# Diff in Diff Analysis practice

In [2]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

%precision %.2f

'%.2f'

In [62]:
df = pd.read_csv('./data/employment.csv')

The dataset is adapted from the dataset in Card and Krueger (1994), which estimates the causal effect of an increase in the state minimum wage on the employment.

On April 1, 1992, New Jersey raised the state minimum wage from 4.25 USD to 5.05 USD while the minimum wage in Pennsylvania stays the same at 4.25 USD. data about employment in fast-food restaurants in NJ (0) and PA (1) were collected in February 1992 and in November 1992. 384 restaurants in total after removing null values The calculation of DID is simple:

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   state          384 non-null    int64  
 1   total_emp_feb  384 non-null    float64
 2   total_emp_nov  384 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 9.1 KB


In [64]:
df.head()

,state,total_emp_feb,total_emp_nov
0,0,40.50,24.00
1,0,13.75,11.50
2,0,8.50,10.50
3,0,34.00,20.00
4,0,24.00,35.50


In [65]:
df.state.value_counts()

1    309
0     75
Name: state, dtype: int64

# New Jersey data

In [6]:
# New Jersey
df_nj = df[df.state == 0]
df_nj['delta'] = df_nj['total_emp_nov'] - df_nj['total_emp_feb']

/var/folders/8c/3nykxypj32j_z4l0d64bln480000gp/T/ipykernel_99260/3770020265.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nj['delta'] = df_nj['total_emp_nov'] - df_nj['total_emp_feb']


In [13]:
print(f'the change of employment from feb to nov in New Jersey is {df_nj.delta.mean():.2f}')

the change of employment from feb to nov in New Jersey is -2.28


# Pennsylvania

In [8]:
df_pa = df[df.state == 1]
df_pa['delta'] = df_pa['total_emp_nov'] - df_pa['total_emp_feb']

/var/folders/8c/3nykxypj32j_z4l0d64bln480000gp/T/ipykernel_99260/3652486956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pa['delta'] = df_pa['total_emp_nov'] - df_pa['total_emp_feb']


In [14]:
print(f'the change of employment from feb to nov in Pennsylvania is {df_pa.delta.mean():.2f}')

the change of employment from feb to nov in Pennsylvania is 0.47


The difference between New Jersey and Pennsylvania is 0.47 - (-2.28) = 2.75. However, how can we know that this is statistically significant? We use the linear regression. 

𝑦 = 𝛽0 + 𝛽1∗state + 𝛽2∗month + 𝛽3∗(interaction) + 𝜀
 
state is 0 for the control group (New Jersey) and 1 for the treatment group (Pennsylvania)
month is 0 for before (Feb) and 1 for after (Nov)
we can insert the values of state and month using the table below and see that coefficient (𝛽3) of the interaction of state and month is the value for DID

In [52]:
df_train = pd.melt(df, id_vars=['state'], value_vars=['total_emp_feb', 'total_emp_nov'], var_name='month', value_name='total_emp')

In [53]:
df_train

,state,month,total_emp
0,0,total_emp_feb,40.50
1,0,total_emp_feb,13.75
2,0,total_emp_feb,8.50
3,0,total_emp_feb,34.00
4,0,total_emp_feb,24.00
...,...,...,...
763,1,total_emp_nov,23.75
764,1,total_emp_nov,17.50
765,1,total_emp_nov,20.50
766,1,total_emp_nov,20.50


In [54]:
df_train.loc[df_train.month == 'total_emp_feb', 'month'] = 0
df_train.loc[df_train.month == 'total_emp_nov', 'month'] = 1
df_train['interaction'] = df_train['state'] * df_train['month']

In [56]:
df_train.state.value_counts()

1    618
0    150
Name: state, dtype: int64

In [59]:
df_train.month.value_counts()

0    384
1    384
Name: month, dtype: int64

In [47]:
df_train.month.value_counts()

0    384
1    384
Name: month, dtype: int64

In [66]:
df_train.interaction.value_counts()

0    459
1    309
Name: interaction, dtype: int64

In [82]:
import statsmodels.formula.api as sm
model = sm.ols('total_emp ~ state + month + interaction', data=df_train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              total_emp   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.947
Date:                Thu, 25 Apr 2024   Prob (F-statistic):              0.121
Time:                        21:22:02   Log-Likelihood:                -2817.6
No. Observations:                 768   AIC:                             5643.
Df Residuals:                     764   BIC:                             5662.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           23.3800      1.098  

The p-value for interaction is not statistically significant, which means that the average total employees per restaurant increased after the minimal salary raise by 2.75 FTE (full-time equivalent) but the result may be just due to random factors.